# imports

In [2]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_feature_generation.SubsamplingFunctions import *
from TCR_feature_generation.TCR_feature_generation_functions import *

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
stop
done
done


In [3]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'18042018'

# clustering trials:

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
dataSetName='PNP515_ss12500_rep2'
dataSetFolder='%s/TCR_real_data/SubSampled12500data_rep2' %MyPath
minNsharedList=[2,50]
metricList=['braycurtis','euclidean','jaccard']

for minNshared in minNsharedList:
    print 'minNshared is %s' %minNshared
    pivot_file='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA' %(dataSetFolder,dataSetName,minNshared)
    pivot=pd.read_pickle(pivot_file)
    print len(pivot)
    for metric in metricList:
        print 'metric=%s' %metric
        clusterInfoFile='%s/sharingAnalysis/clusteringInfo_%s_minNshared%s_%s.xlsx' %(dataSetFolder,
                                                        dataSetName, minNshared,metric)
        
        # generate the linkage matrix
        print 'generating linkage matrix...'
        Z = linkage(pivot, method='single',metric=metric)
        dfZ=pd.DataFrame(Z)
        dfZ=dfZ.rename(columns={0:'sample_id1',1:'sample_id2',2:'distance',3:'nSamplesInCluster'})
        sampleIDdf=pivot.reset_index()['Sample']
        dfZwithSample1=pd.merge(dfZ,pd.DataFrame(sampleIDdf),how='left',left_on='sample_id1',right_index=True)\
        .rename(columns={'Sample':'Sample1'})
        dfZwithSample2=pd.merge(dfZwithSample1,pd.DataFrame(sampleIDdf),how='left',left_on='sample_id2',right_index=True)\
        .rename(columns={'Sample':'Sample2'})
        dfZwithSample2=dfZwithSample2.drop(['sample_id1','sample_id2'],axis=1)
        relFile='%s/Sample files/PNP515-relationships.xlsx' %MyPath
        relDF=pd.read_excel(relFile)
        relDF=relDF[['BD','Relations']]
        dfZwithSamplesAndRelations=pd.merge(dfZwithSample2,relDF,how='left',left_on='Sample1',right_on='BD')\
        .rename(columns={'Relations':'Sample1rel'})
        dfZwithSamplesAndRelations=pd.merge(dfZwithSamplesAndRelations,relDF,how='left',left_on='Sample2',right_on='BD')\
        .rename(columns={'Relations':'Sample2rel'})
        dfZwithSamplesAndRelations=dfZwithSamplesAndRelations.drop(['BD_x','BD_y'],axis=1)
        print dfZwithSamplesAndRelations.head(10)
        dfZwithSamplesAndRelations.to_excel(clusterInfoFile)
        print 'done'
        
        
    

In [ ]:
'''
This (very very briefly) compares (correlates) the actual pairwise distances of all your samples to those 
implied by the hierarchical clustering. The closer the value is to 1, the better the clustering preserves 
the original distances
'''

from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, coph_dists = cophenet(Z, pdist(pivot,metric='braycurtis'))
c

# try to generate distance matrix based on sharing matrix and convert it to redundant table:


In [6]:
#load distance matrix:
datasetFolder='%s/TCR_real_data' %MyPath
datasetName='PNP515'
minNshared=50

sharingMatrixFile='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA' %(datasetFolder,datasetName,
                                                                     minNshared)

sharingDF=pd.read_pickle(sharingMatrixFile)
sharingDF.head()

Sequence,CASRENTEAFF,CASRGETQYF,CASRGGETQYF,CASRGNTEAFF,CASRGQGNTEAFF,CASRGSGANVLTF,CASRGSTDTQYF,CASRISYEQYF,CASRLAGETQYF,CASRLNTEAFF,...,CSAREREYEQYF,CSARGDRDYEQYF,CSARGLAGYNEQFF,CSARGQGDTEAFF,CSARGTGDTEAFF,CSARLADNEQFF,CSARLAGDTGELFF,CSARLAGGQETQYF,CSARQGDTEAFF,CSARTGDTEAFF
Sample,,,,,,,,,,,,,,,,,,,,,
BD1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.013577,0.000000,0.002715,0.005431,...,0.0,0.0,0.000000,0.0,0.002715,0.000000,0.00000,0.0,0.002715,0.00000
BD10,0.002418,0.0,0.002418,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.007255,0.0,0.000000,0.000000,0.01451,0.0,0.000000,0.00000
BD101,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.006098,0.073175,0.00000,0.0,0.000000,0.00000
BD105,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.004106,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.00000
BD106,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.005820,0.00291


In [16]:
print len(sharingDF)

515


In [7]:
from scipy.spatial.distance import pdist

In [35]:
dist_matrix=pd.Series(pdist(sharingDF.fillna(0),'braycurtis'))
dist_matrix

0         0.855614
1         0.916742
2         0.838824
3         0.845979
4         0.844233
5         0.833759
6         0.850109
7         0.822238
8         0.788955
9         0.893178
10        0.850507
11        0.884445
12        0.827022
13        0.890959
14        0.850975
15        0.794732
16        0.855270
17        0.833126
18        0.832842
19        0.859368
20        0.812972
21        0.822799
22        0.836829
23        0.874728
24        0.793777
25        0.811244
26        0.799298
27        0.866695
28        0.825355
29        0.916551
            ...   
132325    0.921065
132326    0.870241
132327    0.871357
132328    0.921164
132329    0.924318
132330    0.878065
132331    0.893321
132332    0.892486
132333    0.918789
132334    0.832911
132335    0.840262
132336    0.789794
132337    0.792089
132338    0.827098
132339    0.861344
132340    0.878715
132341    0.881080
132342    0.848225
132343    0.880213
132344    0.884963
132345    0.869905
132346    0.

In [19]:
515*515/2

132612

In [20]:
distDF=pd.DataFrame(index=sharingDF.index,data=dist_matrix)
#columns=sharingDF.index)
distDF.head()

,0
Sample,
BD1,NaN
BD10,NaN
BD101,NaN
BD105,NaN
BD106,NaN


In [33]:
dist_matrix=pd.Series(dist_matrix)

TypeError: 'numpy.int64' object is not iterable

In [37]:
sample1List=[]
sample2List=[]

count=0
for i in range(len(sharingDF.index)):
    for j in range(i+1,len(sharingDF.index)):
        sample1=sharingDF.index[i]
        sample2=sharingDF.index[j]
        sample1List.append(sample1)
        sample2List.append(sample2)
        count=count+1
print count
df=pd.DataFrame({'sample1':sample1List,'sample2':sample2List,'dist':dist_matrix})
df.sort_values(by='dist')

132355


,dist,sample1,sample2
118499,0.547348,BD705,BD714
109678,0.549595,BD617,BD838
36411,0.551609,BD272,BD574
39879,0.573962,BD3,BD574
41104,0.582277,BD314,BD481
42237,0.585084,BD317,BD928
87682,0.613346,BD481,BD800
42075,0.614162,BD317,BD694
116406,0.628279,BD694,BD928
49205,0.633325,BD349,BD800


In [39]:
file1='%s/Sample files/PNP515-relationships.xlsx' %MyPath
PNP515rel=pd.read_excel(file1)
PNP515rel.head()

,BD,Relations,Relative,is relative in PNP515?,comments,RelGroup,SameGroup,BloodGroup
0,BD12,husband of BD121(568087),BD121,1,NaN,2,1,1
1,BD121,wife of BD12(388380),BD12,1,NaN,2,1,1
2,BD124,husband of BD127(225255),BD127,1,NaN,3,1,1
3,BD127,wife of BD124(345928),BD124,1,NaN,3,1,1
4,BD146,wife of BD881(595162),BD881,1,NaN,4,1,1


In [45]:
merged=pd.merge(df,PNP515rel[['BD','Relations']],how='left',left_on='sample1',right_on='BD')
merged=merged.rename(columns={'Relations':'sample1 relations'})
merged=pd.merge(merged,PNP515rel[['BD','Relations']],how='left',left_on='sample2',right_on='BD' )
merged=merged.rename(columns={'Relations':'sample2 relations'})
merged=merged.drop(['BD_x','BD_y'],axis=1)
merged.sort_values(by='dist')

,dist,sample1,sample2,sample1 relations,sample2 relations
118499,0.547348,BD705,BD714,same person as BD714,same person as BD705
109678,0.549595,BD617,BD838,The same person as BD838. wife of BD615/BD839(...,The same person as BD617. wife of BD615/BD839(...
36411,0.551609,BD272,BD574,NaN,husband of BD571(244911)
39879,0.573962,BD3,BD574,NaN,husband of BD571(244911)
41104,0.582277,BD314,BD481,NaN,NaN
42237,0.585084,BD317,BD928,NaN,NaN
87682,0.613346,BD481,BD800,NaN,wife of BD828(518189)
42075,0.614162,BD317,BD694,NaN,NaN
116406,0.628279,BD694,BD928,NaN,NaN
49205,0.633325,BD349,BD800,"sister of BD352(555992), mother of BD347(95894...",wife of BD828(518189)


# develop a function - heatmap and clustermap for a pivot table:

In [ ]:
def gen_heatmap_and_clustermap(Rtable,RtableName,figFolder,figsizeX,figsizeY,fontsizeX,fontsizeY,fontSizeLabel,sortMethod):

In [ ]:
RtableName='TRIAL'
figFolder='%s/TCR_real_data/SubSampled12500data_rep2/sharingAnalysis' %MyPath
figsizeX=30
figsizeY=30
fontsizeX=5
fontsizeY=5
fontSizeLabel=20
sortMethod='max'

In [ ]:
file1='%s/TCR_real_data/SubSampled12500data_rep2/sharingAnalysis/sharingMatrix_PNP515_ss12500_rep2_minNshared50_RA' %MyPath
pivot=pd.read_pickle(file1)
sortMethod='max'
pivot.head()

In [ ]:
print pivot.shape
if sortMethod=='max':
    pivot[sortMethod]=pivot.max(axis=1)
    pivot.loc[sortMethod,:]=pivot.max(axis=0)
elif sortMethod=='mean':
    pivot[sortMethod]=pivot.mean(axis=1)
    pivot.loc[sortMethod,:]=pivot.mean(axis=0)
pivot=pivot.sort_values(by=sortMethod,axis=1)
pivot=pivot.sort_values(by=sortMethod,axis=0)
pivot=pivot.drop(sortMethod,axis=1)
pivot=pivot.drop(sortMethod,axis=0)
pivot.head()

In [ ]:
#get color list:

colorFile='%s/Sample files/PNP515-relationships.xlsx' %MyPath
colorDF=pd.read_excel(colorFile)
colorDF=colorDF[['BD','RelGroup','SameGroup','BloodGroup']]
colorDF.head()

In [ ]:
chosenColorDF=colorDF[['BD','SameGroup']].set_index('BD')


In [ ]:



uniqueSpecies=pivot.index.unique()
genus=[x.split('_')[0] for x in uniqueSpecies ]
genus2=list(set(genus))
genus2n=len(genus2)
if genus2n<10:
    fact=0.1
elif genus2n<20:
    fact=0.05
elif genus2n<50:
    fact=0.02
else:
    fact=0.01 
species_genus_df=pd.DataFrame({'species':uniqueSpecies,'genus':genus})
genus2color=[]
for n, g in enumerate(genus2):
    color=plt.cm.get_cmap('hsv')(fact*n)
    genus2color.append(color)
genus2colorDF=pd.DataFrame({'genus':genus2,'color':genus2color})
speciesGenusColorDF=pd.merge(species_genus_df,genus2colorDF,how='left', left_on='genus',right_on='genus')
cluster_colors=list(speciesGenusColorDF['color'])

In [ ]:
print 'generate heatmap...'
heatMapFig, ax = plt.subplots(figsize=(figsizeX,figsizeY))         # Sample figsize in inches
sns.heatmap(pivot,ax=ax,vmax=0.5)
ax.yaxis.label.set_size(fontSizeLabel)
ax.tick_params(axis='y', labelsize=fontsizeY)
ax.xaxis.label.set_size(fontSizeLabel)
ax.tick_params(axis='x', labelsize=fontsizeX)
heatMapName='%s_heatMap_size%s-%s_%s' %(RtableName,figsizeX,figsizeY,sortMethod)
heatMapFig.savefig(heatMapName,dpi=200)

plt.show()

In [ ]:
#generate clustermap
print 'generate clustermap...'
cg= sns.clustermap(pivot,figsize=(figsizeX,figsizeY),col_cluster=False,metric='braycurtis')
#,row_colors=cluster_colors)
plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), fontsize=fontsizeY,rotation=0)
plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), fontsize=fontsizeX)
clusterMapName='%s_clusterMap_size%s-%s_%s' %(RtableName,figsizeX,figsizeY,sortMethod)
file1='%s/%s' %(figFolder,clusterMapName)
cg.savefig(file1,dpi=200)
plt.show()

In [ ]:



#generate heatmap:
print 'generate heatmap...'
heatMapFig, ax = plt.subplots(figsize=(figsizeX,figsizeY))         # Sample figsize in inches
sns.heatmap(pivot,ax=ax)
ax.yaxis.label.set_size(fontSizeLabel)
ax.tick_params(axis='y', labelsize=fontsizeY)
ax.xaxis.label.set_size(fontSizeLabel)
ax.tick_params(axis='x', labelsize=fontsizeX)
heatMapName='%s_heatMap_size%s-%s_%s' %(RtableName,figsizeX,figsizeY,sortMethod)
heatMapFig.savefig(heatMapName,dpi=200)

#generate clustermap
print 'generate clustermap...'
cg= sns.clustermap(pivot,figsize=(figsizeX,figsizeY),col_cluster=True,row_colors=cluster_colors)
plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), fontsize=fontsizeY,rotation=0)
plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), fontsize=fontsizeX)
clusterMapName='%s_clusterMap_size%s-%s_%s' %(RtableName,figsizeX,figsizeY,sortMethod)
cg.savefig(clusterMapName,dpi=200)

print 'done'

return heatMapFig,cg